In [ ]:
import os
import sys
sys.path.append(os.path.abspath("../"))

from dotenv import find_dotenv, load_dotenv
from funes.agents.agent_types import Role, Persona, AutogenAgentType
from llm_foundation import logger
from llm_foundation.basic_structs import Provider, LMConfig
from pprint import pprint

import autogen

load_dotenv(find_dotenv())

openai_api_key = os.environ["OPENAI_API_KEY"]

lm_config = LMConfig(model="gpt-4o-mini", provider=Provider.Autogen)
llm_config = lm_config.to_autogen()
llm_config

In [ ]:
francisco = Persona.from_json_file("Persona/Francisco.json")
neo4j_persona = Persona.from_json_file("Persona/Neo4jExpert.json")
print(francisco, neo4j_persona)

In [3]:
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector

NEO4J_URI = os.environ["NEO4J_URI"]
NEO4J_USERNAME = os.environ["NEO4J_USERNAME"]
NEO4J_PASSWORD = os.environ["NEO4J_PASSWORD"]
NEO4J_DATABASE = "neo4j"

In [ ]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

print(type(kg))

In [ ]:
kg.get_structured_schema



In [ ]:
query = """
CREATE (a:Author {name: 'Sebastian Raschka'}),\n       (b:Book {title: 'Build A Large Language Model (From Scratch)', year: '2024', isbn: '978-1633437166', url: 'https://www.manning.com/books/build-a-large-language-model-from-scratch'}),\n       (p:Publisher {name: 'Manning'}),\n       (a)-[:WROTE]->(b),\n       (b)-[:PUBLISHED_BY]->(p);\n
"""

result = kg.query(query)
result

In [ ]:


from abc import ABC
from typing import List, Optional
from langchain_community.graphs.neo4j_graph import Neo4jGraph


class GraphMetadata(ABC):
    
    def __init__(self, graph: Neo4jGraph):
        self.graph = graph
        self.schema = self.graph.get_structured_schema

    def get_node_names(self) -> List[str]:
        nodes = self.schema["node_props"]
        return list(nodes.keys())

    def get_edge_names(self, origin: Optional[str] = None, dest: Optional[str] = None):
        
        def filter_edges_on_prop(edges, prop_id, value):
            return [edge for edge in edges if edge[prop_id] == value]
        
        nodes = self.schema["relationships"]
        
        if origin is not None:
            nodes = filter_edges_on_prop(nodes, "start", origin)

        if dest is not None:
            nodes = filter_edges_on_prop(nodes, "end", dest)
        
        return [node['type'] for node in nodes], nodes

    def is_node_in_graph(self, node: str):
        nodes = self.get_node_names()
        return node in nodes

    def get_node_attributes_from_node(self, node):
        attributes = []
        if self.is_node_in_graph(node):
            attributes = [property_info["property"] for property_info in self.schema["node_props"][node]]
        return attributes

    def get_node_instance(self, node: str, instance_id: str, instance_name: str):

        res = ""        
        if self.is_node_in_graph(node):
            query_node_id = f"{node.lower()}"
            res = self.graph.query(f"""
                          MATCH ({query_node_id}:{node}) 
                          WHERE {query_node_id}.{instance_id} = '{instance_name}' 
                          RETURN {query_node_id}
                          """)
        return res

graph_metadata = GraphMetadata(kg)

from langchain.tools import BaseTool
class GetNodeNames(BaseTool):
    name = "get_node_names"
    description = "Extract node names from a graph"
    # args_schema: Type[BaseModel] = SearchToolInput
    graph: GraphMetadata

    def _run(self):
        return self.graph.get_node_names()
        

get_node_names = GetNodeNames(graph=graph_metadata)

# Get graph nodes 
print("Graph Nodes")
print(graph_metadata.get_node_names())
print(graph_metadata.get_node_attributes_from_node("Person"))

# Get edges
print("Graph Edges")
print(f"All: {graph_metadata.get_edge_names()}")
print(f"Person-as-origin: {graph_metadata.get_edge_names('Person')}")
print(f"Person-as-end: {graph_metadata.get_edge_names(None, 'Person')}")
print(f"Book-as-origin: {graph_metadata.get_edge_names('Book')}")
print(f"Book-as-end: {graph_metadata.get_edge_names(None, 'Book')}")
print(f"Person-Book: {graph_metadata.get_edge_names('Person', 'Book')}")

# Get graph node instances
print("Graph Francisco Node Instances")
print(f"All instances: {graph_metadata.get_node_instance('Person', 'name', 'Francisco')}")

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_function

get_node_names = GetNodeNames(graph=graph_metadata)

get_node_names_tool = convert_to_openai_function(get_node_names)

tools = [get_node_names_tool]
llm_with_tools_config = llm_config.copy()
llm_with_tools_config.update({"functions": tools})

# neo4j_tool_wishperer = neo4j_persona.role_to_autogen_agent("neo4j_tool_whisperer", AutogenAgentType.AssistantAgent, llm_config=llm_with_tools_config)
# # neo4j_tool_wishperer.register_for_llm(name="get_node_names", description="Extract node names from a graph")(get_node_names._run)
# neo4j_tool_wishperer.register_function(
#     function_map={
#         get_node_names.name: get_node_names._run
#     })


neo4j_agent = neo4j_persona.role_to_autogen_agent("neo4j", AutogenAgentType.AssistantAgent, llm_config=llm_with_tools_config)
# neo4j_agent.register_for_execution(name="get_node_names")(get_node_names._run)
neo4j_agent.register_function(
    function_map={
        get_node_names.name: get_node_names._run
    })


francisco_learner = francisco.role_to_autogen_agent("learner", AutogenAgentType.UserProxyAgent, "NEVER", llm_config=llm_config, termination_function=lambda msg: "terminate" in msg["content"].lower(),)

from autogen.agentchat import GroupChat, GroupChatManager

agent_speaker_transitions_dict = {
    francisco_learner: [neo4j_agent],
    neo4j_agent: [francisco_learner],
    # neo4j_tool_wishperer: [neo4j_agent]
}

groupchat = GroupChat(
    agents = [neo4j_agent, francisco_learner],
    messages=[],
    max_round=10,
    select_speaker_auto_verbose=True,
    speaker_transitions_type="allowed",  # This has to be specified if the transitions below apply
    allowed_or_disallowed_speaker_transitions=agent_speaker_transitions_dict,
)

manager = GroupChatManager(
    groupchat=groupchat, 
    llm_config=llm_config,
    system_message="You act as a coordinator for different specialiced roles. If you don't have anything to say, just say TERMINATE."
)        


book_json = {
    "author": "Sebastian Raschka",
    "title": "Build A Large Language Model (From Scratch)",
    "publisher": "Manning",
    "year": "2024",
    "isbn": "978-1633437166",
    "url": "https://www.manning.com/books/build-a-large-language-model-from-scratch"
}


content = f"""
Get a Cypher query to encode the information on the following json object. Reply with the query without adding anything else.

{book_json}
"""

response = francisco_learner.initiate_chat(
    manager,
    message={"content": content, "role": "user"},
)


def find_last_message(name: str, chat_history):
    for message in reversed(chat_history):
        if message["name"] == name:
            return message
    return None

print(find_last_message("neo4j", response.chat_history)["content"])


# francisco_learner.initiate_chat(neo4j_agent, message="")

In [ ]:

book_json = {
    "author": "Sebastian Raschka",
    "title": "Build A Large Language Model (From Scratch)",
    "publisher": "Manning",
    "year": "2024",
    "isbn": "978-1633437166",
    "url": "https://www.manning.com/books/build-a-large-language-model-from-scratch"
}


content = f"""
Given the following json object about a book release, identify the main entities and relationships to build a graph encoded with nodes and relationships
with Cypher language. Name all nodes and relationships (e.g. MERGE (a)-[wrote:WROTE]->(b)). Check also first if the graph contains a high level entity 
for some of the new entities identified and try to integrate better the new subgraph (e.g. if the graph has a Person node integrate an Author as a person
that has a property author in a relationship WROTE). Reply only with the Cypher queries and RETURN statements with the affected 
nodes and relationships, but without any wrappers nor bat-ticks.

{book_json}
"""

print(content)

resp = francisco_learner.generate_reply(
    messages=[{"content": content, "role": "user"}]
)

pprint(resp)


In [ ]:
print(resp)

In [ ]:
content = f"""
Given the following Cypher query, identify if the current graph contains a high level entity 
for some of the new entities identified in the query and try to use them in the new subgraph (e.g. check if the graph has a IndividualContributor node 
and in the query there has been identified an Boss, rewrite the query to use Person as node and integrate the author as a property
of a possible relationship MANAGES). Name all nodes and relationships (e.g. MERGE (a)-[wrote:WROTE]->(b)) Reply only with the rewriten Cypher queries and RETURN statements with the affected nodes and 
relationships, but without any wrappers nor bat-ticks.

{resp}
"""

print(content)

resp_rewrite = francisco_learner.generate_reply(
    messages=[{"content": content, "role": "user"}]
)

pprint(resp_rewrite)


In [ ]:
result = kg.query(resp)
result